# Introduction/Business Problem

To find a good location for opening a motel, with maximum footprint of tourists. A great business hotel ensures that you have everything you need on site to stay connected and productive. Complimentary Wi-Fi, in-room phones, and a 24/7 business center are essential, and extra space to work makes it that much easier to stay focused.

Now that all your prep is done, don’t stress about making it to your morning meeting. Make sure that your hotel has a great location that makes it easy to get to where you need to be. Clarion Inn Elmhurst – Oakbrook is ideally located in the Chicago suburbs, close to local corporations like Hub Group, Master Lock, Kellogg, and more, while also boasting just a quick drive to downtown and O’Hare Airport.

# Data

i will use distance from tourist spots and other hotels as a factors. Also number of vistors for different locations such as local markets etc. like Grand Motel, 1 stars, Scarborough, Toronto which is situated 21 km from The Distillery District, 22 km from St. Lawrence Market and 23 km from Hockey Hall of Fame.

In [2]:

#!conda install -c conda-forge folium=0.5.0 --yes # comment/uncomment if not yet installed.
!conda install -c conda-forge geopy --yes        # comment/uncomment if not yet installed

# Import necessary library
import json
import pandas as pd
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

In [3]:
address = 'New delhi'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

28.6141793 77.2022662


In [4]:
CLIENT_ID = 'NFK1ZGILDEKZCFJKZAOZNGQILQABON3Q2NVABLR2XMLX15U1' # your Foursquare ID
CLIENT_SECRET = 'MBLEBI03ILADL0JFLEL3BKKTAEQC5KZ2VACN1V3ZNY024DSL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NFK1ZGILDEKZCFJKZAOZNGQILQABON3Q2NVABLR2XMLX15U1
CLIENT_SECRET:MBLEBI03ILADL0JFLEL3BKKTAEQC5KZ2VACN1V3ZNY024DSL


In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
search_query = 'it'
radius = 50000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))
# assign relevant part of JSON to venues
items = results['response']['groups'][0]['items']
items[0]

# tranform venues into a dataframe
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,The Imperial,Hotel,Janpath,28.625548,77.218664,"[{'label': 'display', 'lat': 28.62554803815942...",2041,110001,IN,New Delhi,Delhi,India,"[Janpath, New Delhi 110001, Delhi, India]",NaN,NaN,4bc8f2c7762beee1a8bb3d38
1,Tamra,Restaurant,"19, Ashoka Rd",28.620543,77.218174,"[{'label': 'display', 'lat': 28.62054318423011...",1708,110001,IN,New Delhi,Delhi,India,"[19, Ashoka Rd (Windsor Pl), New Delhi 110001,...",Windsor Pl,NaN,54dc85c7498ef8f9ab9b3c08
2,Pandey Paan,Smoke Shop,"Shop No 3, North Avenue",28.622249,77.201075,"[{'label': 'display', 'lat': 28.6222491879439,...",905,NaN,IN,New Delhi,Delhi,India,"[Shop No 3, North Avenue (MP Market), New Delh...",MP Market,NaN,4f8c4443e4b0a95f9685c9f6
3,The Leela Palace,Hotel,"Chanakyapuri, Diplomatic Enclave",28.579923,77.189291,"[{'label': 'display', 'lat': 28.57992320896024...",4018,NaN,IN,New Delhi,Delhi,India,"[Chanakyapuri, Diplomatic Enclave, New Delhi, ...",NaN,NaN,4cfe905066ee9eb038e2a64e
4,Indian Accent,Indian Restaurant,Lodhi Road,28.592026,77.238006,"[{'label': 'display', 'lat': 28.59202584198747...",4275,110003,IN,New Delhi,Delhi,India,"[Lodhi Road, New Delhi 110003, Delhi, India]",NaN,NaN,5a0557269deb7d67455194ac
5,Naturals Ice Cream,Ice Cream Shop,L Block Outer Circle,28.634455,77.222139,"[{'label': 'display', 'lat': 28.63445530551318...",2977,NaN,IN,New Delhi,Delhi,India,"[L Block Outer Circle (Connaught Place), New D...",Connaught Place,NaN,54783eab498e910f8bd1781d
6,Lodhi Gardens (लोधी बाग़) (Lodhi Gardens),Park,Lodhi Rd,28.591424,77.220899,"[{'label': 'display', 'lat': 28.59142408796874...",3119,NaN,IN,New Delhi,Delhi,India,"[Lodhi Rd, New Delhi, Delhi, India]",NaN,NaN,4c025c949a7920a1de4fce79
7,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636,"[{'label': 'display', 'lat': 28.60068629868818...",2898,110048,IN,New Delhi,Delhi,India,"[68A (Khan Market), New Delhi 110048, Delhi, I...",Khan Market,NaN,4b701cbaf964a5209b072de3
8,ITC Maurya,Hotel,"Diplomatic Enclave, Sardar Patel Marg",28.597130,77.173643,"[{'label': 'display', 'lat': 28.59713, 'lng': ...",3380,110021,IN,New Delhi,Delhi,India,"[Diplomatic Enclave, Sardar Patel Marg, New De...",NaN,NaN,4b78c1faf964a52016e12ee3
9,The Lodhi Hotel,Hotel,Lodhi Road,28.591669,77.238131,"[{'label': 'display', 'lat': 28.59166874719434...",4308,110 003,IN,New Delhi,Delhi,India,"[Lodhi Road, New Delhi 110 003, Delhi, India]",NaN,NaN,4bd78af3304fce7284b033ab


In [34]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [48]:
hotels = dataframe_filtered[['name','address','lat','lng','distance','categories']]

hotel = hotels[hotels['categories'] == 'Hotel']
hotel.columns =['Name','Address','Latitude','Longitude','Distance','Categories'] 
hotel.drop('Categories',axis = 1,inplace = True)
hotel

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Name,Address,Latitude,Longitude,Distance
0,The Imperial,Janpath,28.625548,77.218664,2041
3,The Leela Palace,"Chanakyapuri, Diplomatic Enclave",28.579923,77.189291,4018
8,ITC Maurya,"Diplomatic Enclave, Sardar Patel Marg",28.597130,77.173643,3380
9,The Lodhi Hotel,Lodhi Road,28.591669,77.238131,4308
15,The Oberoi,Dr. Zakir Hussain Marg,28.596564,77.239634,4145
16,The Claridges,Aurangzeb Rd New Delhi New Delhi Delhi India,28.600705,77.216601,2052
23,Taj Palace Hotel,Sardar Patel Marg,28.595098,77.170913,3728


In [53]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(hotel.Latitude, hotel.Longitude, hotel.Distance):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup= str(label),
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map



# As we can see chanakyapuri is best place falling equally close to central delhi and airport